In [0]:
import requests

pharm_class_epc = "nonsteroidal+anti-inflammatory+drug"
limit  = 100

# can add parameters while scheduling the job
url = "https://api.fda.gov/drug/event.json?search=patient.drug.openfda.pharm_class_epc:\"{}\"&limit={}".format(pharm_class_epc,limit)
print(url)
response = requests.get(url)

# Check the status code to ensure the request was successful
if response.status_code == 200:
    data = response.json()
else:
    print("Failed to fetch data")

https://api.fda.gov/drug/event.json?search=patient.drug.openfda.pharm_class_epc:"tumor necrosis factor blocker"&limit=100

In [0]:
raw_data = data.get('results')

In [0]:
raw_data_df = spark.createDataFrame(raw_data)

In [0]:
raw_data_df.write.format("delta").mode("append").option("overwriteSchema", "true").option("path","/bronze_layer/adverse_events").saveAsTable("adverse_events_bz")

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1751324483489061> in <module> 
 ----> 1 raw_data_df . write . format ( "delta" ) . mode ( "append" ) . option ( "overwriteSchema" , "true" ) . option ( "path" , "/bronze_layer/adverse_events" ) . saveAsTable ( "adverse_events_bz" ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in saveAsTable (self, name, format, mode, partitionBy, **options) 
 1183 if format is not None : 
 1184 self . format ( format ) 
 -> 1185 self . _jwrite . saveAsTable ( name ) 
 1186 
 1187 def json(self, path, mode=None, compression=None, dateFormat=None, timestampFormat=None,

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o395.saveAsTable.
: java.security.AccessControlException: Permission denied: user [kraghu491] does not have [read] privilege on [dbfs:/bronze_layer/adverse_events]
	at com.privacera.spark.authorizer.t.a(t.java)
	at com.privacera.spark.agent.authorizer.PrivaceraSparkAuthorizer.authorize(PrivaceraSparkAuthorizer.java)
	at com.privacera.spark.agent.authorizer.c.a(c.java)
	at com.privacera.spark.agent.r.a(r.java)
	at com.privacera.spark.base.interceptor.b.a(b.java)
	at com.privacera.spark.agent.p.a(p.java)
	at com.privacera.spark.agent.p.apply(p.java)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$3(RuleExecutor.scala:221)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:80)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$2(RuleExecutor.scala:221)
	at scala.collection.LinearSeqOptimized.foldLeft(LinearSeqOptimized.scala:126)
	at scala.collection.LinearSeqOptimized.foldLeft$(LinearSeqOptimized.scala:122)
	at scala.collection.immutable.List.foldLeft(List.scala:89)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$1(RuleExecutor.scala:218)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$1$adapted(RuleExecutor.scala:210)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.execute(RuleExecutor.scala:210)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$executeAndTrack$1(RuleExecutor.scala:188)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker$.withTracker(QueryPlanningTracker.scala:142)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.executeAndTrack(RuleExecutor.scala:188)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$optimizedPlan$1(QueryExecution.scala:112)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:80)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker.measurePhase(QueryPlanningTracker.scala:300)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$executePhase$1(QueryExecution.scala:180)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:854)
	at org.apache.spark.sql.execution.QueryExecution.executePhase(QueryExecution.scala:180)
	at org.apache.spark.sql.execution.QueryExecution.optimizedPlan$lzycompute(QueryExecution.scala:109)
	at org.

In [0]:
df2 = raw_data_df.select("*",raw_data_df["patient.patientonsetageunit"].alias("patient_patientonsetageunit"),raw_data_df["patient.patientsex"].alias("patient_patientsex"),raw_data_df["patient.reaction"].alias("patient_reaction"),raw_data_df["patient.patientonsetage"].alias("patient_patientonsetage"),raw_data_df["patient.drug"].alias("patient_drug"),raw_data_df["primarysource.qualification"].alias("qualification"))

In [0]:
df_final = df2.drop("patient").drop("primarysource")

In [0]:
# for the first run of the job
df_final.write.format("delta").mode("append").partitionBy("qualification,receivedate").option("overwriteSchema", "true").option("path","/Goldlayer/adverse_events").saveAsTable("adverse_events")

#from second run of the job
deltatable = DeltaTable.forPath(spark, '/Goldlayer/adverse_events')
deltatable.alias("target").merge(source = df_final.alias("new"),condition = "target.qualification = new.qualification and target.safetyreportid = new.safetyreportid").whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

In [0]:
%sql
spark.sql("OPTIMIZE '/Goldlayer/adverse_events' ZORDER BY (safetyreportid)")